In [8]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os
from matplotlib import pyplot as plt
import io
from google.cloud import vision
from PIL import Image
import json




Using TensorFlow backend.


In [8]:
path ='./image.jpg'

In [9]:
# 이미지를 로드합니다
img = cv2.imread(path)

In [10]:
# 분류를 위한 이미지 전처리를 수행합니다
image = cv2.resize(img, (280, 160))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

# 학습된 네트워크와 `MultiLabelBinarizer`를 로드합니다
print("[INFO] loading network...")
model = load_model("model.h5")
mlb = pickle.loads(open("labelbin", "rb").read())

# 이미지에 대한 분류를 수행한 후, 
# 확률이 가장 높은 두 개의 클래스 라벨을 찾습니다
# print("[INFO] classifying image...")
proba = model.predict(image)[0]
idxs = np.argsort(proba)[::-1][:4]

[INFO] loading network...


In [63]:
# 나올 수 있는 모든 라벨들을 출력합니다
for (i, label) in enumerate(mlb.classes_):
	print("{}. {}".format(i + 1, label))

1. a hemicyclea semicircle
2. black
3. blue
4. blue, light
5. blue, transparency
6. bluish green
7. bluish green, transparency
8. brown
9. brown, transparency
10. circle
11. dark blue
12. dark blue, transparency
13. diamond
14. ellipse
15. gray
16. green
17. green, transparency
18. hexagon
19. octagon
20. orange
21. orange, transparency
22. pentagon
23. pink
24. pink, deep
25. pink, light
26. pink, transparency
27. purple
28. purple, transparency
29. rectangle
30. red
31. red, transparency
32. tetragon
33. transparency
34. triangle
35. white
36. white, blue
37. white, brown
38. white, green
39. white, red
40. white, transparency
41. white, yellow
42. wine
43. wine, transparency
44. yellow
45. yellow, transparency
46. yellowish green
47. yellowish green, transparency


In [11]:
idxs

array([ 9, 14, 34, 28], dtype=int64)

In [12]:
for (i, j) in enumerate(idxs):
    label = "{}: {:.2f}%".format(mlb.classes_[j], proba[j] * 100)
    


In [6]:
# 각 라벨에 대한 확률을 출력합니다
for (label, p) in zip(mlb.classes_, proba):
	print("{}: {:.2f}%".format(label, p * 100))

a hemicyclea semicircle: 0.00%
black: 0.00%
blue: 0.00%
blue, light: 0.00%
blue, transparency: 0.00%
bluish green: 0.00%
bluish green, transparency: 0.00%
brown: 0.00%
brown, transparency: 0.00%
circle: 2.06%
dark blue: 0.00%
dark blue, transparency: 0.00%
diamond: 0.00%
ellipse: 0.19%
gray: 0.00%
green: 99.87%
green, transparency: 0.14%
hexagon: 0.00%
octagon: 0.00%
orange: 0.00%
orange, transparency: 0.00%
pentagon: 0.00%
pink: 0.00%
pink, deep: 0.00%
pink, light: 0.00%
pink, transparency: 0.00%
purple: 0.00%
purple, transparency: 0.00%
rectangle: 39.29%
red: 0.00%
red, transparency: 0.00%
tetragon: 0.00%
transparency: 0.00%
triangle: 0.00%
white: 0.00%
white, blue: 0.00%
white, brown: 0.00%
white, green: 0.00%
white, red: 0.00%
white, transparency: 0.00%
white, yellow: 0.00%
wine: 0.00%
wine, transparency: 0.00%
yellow: 0.00%
yellow, transparency: 0.00%
yellowish green: 0.03%
yellowish green, transparency: 0.00%


In [13]:
first_acc=mlb.classes_[idxs[0]]
second_acc=mlb.classes_[idxs[1]]
third_acc=mlb.classes_[idxs[2]]
fourth_acc=mlb.classes_[idxs[3]]

print(first_acc)
print(second_acc)
print(third_acc)
print(fourth_acc)

circle
gray
white
rectangle


In [65]:
tablet_shape=[]
tablet_color=[]

tablet_shape_labels=['a hemicyclea semicircle','circle','diamond','ellipse','hexagon','octagon','pentagon',
'rectangle','tetragon','triangle']
tablet_color_labels=['black','blue','blue, light','blue, transparency','bluish green','bluish green, transparency','brown',
' brown, transparency','dark blue','dark blue','transparency','gray','green','green, transparency',
'orange','orange, transparency','pink','pink, deep','pink, light','pink, transparency','purple',
'purple, transparency','red','red, transparency','transparency','white','white, blue','white, brown',
'white, green','white, red','white, transparency','white, yellow','wine','wine, transparency','yellow',
'yellow, transparency','yellowish green','yellowish green, transparency']


In [66]:
for idx in range(4) :
    if mlb.classes_[idxs[idx]] in tablet_shape_labels :
        tablet_shape.append(mlb.classes_[idxs[idx]])
        
    elif mlb.classes_[idxs[idx]] in tablet_color_labels :
        tablet_color.append(mlb.classes_[idxs[idx]])
        

In [67]:


os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ='./crucial-lyceum-274205-eadd5d760574.json'

client = vision.ImageAnnotatorClient()

with io.open('./image.jpg', 'rb') as image_file:    
    content = image_file.read()
    
image = vision.types.Image(content=content)

response = client.text_detection(image=image)
texts = response.text_annotations

search_word=texts[1].description

In [12]:
tablet_shape=['circle']
tablet_color=['green']
search_word='DWP'

In [2]:
import pandas as pd
df1=pd.read_csv('./공공데이터개방_낱알식별목록_re.csv',encoding = 'cp949')

In [13]:
check1 = df1[(df1['의약품제형']==tablet_shape[0]) & (df1['색상앞']==tablet_color[0])]  

In [14]:
check1

,품목일련번호,품목명,업소일련번호,업소명,성상,큰제품이미지,표시앞,표시뒤,의약품제형,색상앞,...,품목허가일자,제형코드명,표기내용앞,표기내용뒤,표기이미지앞,표기이미지뒤,표기코드앞,표기코드뒤,변경일자,이미지이름
139,197500186,에트라빌10밀리그램정(아미트리프틸린염산염),19570006,동화약품(주),이 약은 청색의 제피정이다.,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,D-W,E,circle,blue,...,19750211,필름코팅정,NaN,NaN,NaN,NaN,NaN,NaN,20190518.0,NaN
140,198600437,동화디트로판정(옥시부티닌염산염),19570006,동화약품(주),담청색의 circle정제.,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,D-W,DT,circle,blue,...,19861120,나정,NaN,NaN,NaN,NaN,NaN,NaN,20160525.0,NaN
203,198900344,하이메틴정300밀리그람(시메티딘),19540006,일동제약(주),엷은 청색의 circle 필름코팅정이다.,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,HIMETIN,300,circle,blue,...,19891221,필름코팅정,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,196200046,트리민정4밀리그람(페르페나진),19620008,영진약품(주),청색의 circle정제,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,YJP,NaN,circle,blue,...,19621028,당의정,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278,199200144,스부롤정(부스피론염산염),19860001,한국휴텍스제약(주),청색의 circle 정제,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,HT,SR,circle,blue,...,19920831,나정,NaN,NaN,NaN,NaN,NaN,NaN,20130821.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22439,201905280,아씨엠정200mg(아시클로버),19550012,(주)마더스제약,연청색의 circle정제,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,MTS,4,circle,blue,...,20190725,나정,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22473,201908117,조이락스정(아시클로버),20182522,주식회사 더유제약,연청색의 circle 정제,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,TUP,200,circle,blue,...,20191127,나정,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22562,201906559,프로핀카정(피나스테리드),20180404,주식회사다나젠,청색의 circle필름코팅정,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,DNN,십자,circle,blue,...,20190919,필름코팅정,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22612,201908144,아보리드정0.5밀리그램(두타스테리드),19950015,한국프라임제약(주),연한 파란색의 circle 필름코팅정제,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,AV,NaN,circle,blue,...,20191128,필름코팅정,NaN,NaN,NaN,NaN,NaN,NaN,20191231.0,NaN


In [15]:
check1 = df1[(df1['의약품제형']==tablet_shape[0]) & (df1['색상앞']==tablet_color[0])]  
check2 = check1[(check1['표시앞']==search_word) | (check1['표시뒤']==search_word)]

In [16]:
print(len(check2))

0


In [10]:
check2.to_json('test.json', orient='table')

with open('test.json') as json_file:
    json_data = json.load(json_file)

In [82]:
tablet_name = json_data["data"][0]["품목명"]
tablet_function = json_data["data"][0]["분류명"]

In [83]:
print(tablet_name)
print(tablet_function)

원폴민정(메트포르민염산염)
당뇨병용제
